In [1]:
def ScrapearP12():
  '''
  Scrapeo de informacion de todas las notas de todas las secciones
  de la pagina del diario "Pagina12".

  Precondicion: tener instalados e importados los siguientes modulos: request, bs4, pandas, os
  y datetime. Ademas, se necesita que la pagina web de Pagina no cambie su codigo
  html luego de la fecha: 03/01/2021
  Poscondicion: Guarda en el directorio un archivo csv y devuelve un objeto de 
  tipo DataFrame con la informacion scrapeada.
  '''

  url = 'https://www.pagina12.com.ar/'
  try:
    p12 = requests.get(url)
  except Exception as e:
    print("error en la request")
    print(e)
    print('\n')
    return None

  if p12.status_code != 200:
    print(f"Error en request a p12: {url}")
    print(f"status coder = {p12.status_code}")
    return None

  try:
    p12 = BeautifulSoup(p12.text, 'html')
    secciones = p12.find('ul', class_='horizontal-list main-sections hide-on-dropdown').find_all('li')
    links_secciones = [seccion.find('a').get('href') for seccion in secciones]
  except Exception:
    print("Pagina12 ha cambiado su codigo html, se necesita actualizar el script.")
    return None

  notas = []
  for link in links_secciones:
    notas.extend(links_noticias(link))

  data = []
  for i, nota in enumerate(notas):
    print(f"Scrapeando nota {i}/{len(notas)}")
    data.append(scrape_nota(nota))
  print("Scrapeo finalizado")
  
  df = pd.DataFrame(data)
  folder = "NotasP12_" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  df.to_csv(folder+".csv")

  return df

In [2]:
def links_noticias(url_seccion):
  
  '''
  Dada una url de seccion regresa todos los links de dicha seccion

  Precondicion: ingresar como unico parametros un objeto string con la url de
  la seccion de pagina12
  Poscondicion: devuelve una lista con todos los links de la seccion indicada.
  '''

  try:
    sec = requests.get(url_seccion)
  except Exception as e:
    print("error en la request")
    print(e)
    print('\n')
    return None
  
  if sec.status_code != 200:
    print(f"Error obteniendo notas de la seccion: {urlseccion}")
    print(f"status coder = {sec.status_code}")
    return None
  
  s_seccion = BeautifulSoup(sec.text, 'html')

  links = []

  h2 = s_seccion.find_all('h2', class_='title-list')
  if h2:
     links.append(h2[0].a.get('href'))
  
  h3 = s_seccion.find_all('h3', class_='title-list')
  if h3:
    for h in h3:
      if h.a:
        links.append(h.a.get('href'))
  
  h4 = s_seccion.find_all('h4', class_='is-display-inline title-list')
  if h4:
    for h in h4:
      if h.a:
        links.append(h.a.get('href'))


  return links


In [3]:
def scrape_nota(url_nota):
  '''
  Dada una url de una nota regresa la informacion scrapeada de la misma.

  Precondicion: ingresar como unico parametros un objeto string con la url de
  la nota.
  Poscondicion: devuelve un diccionario con la informacion de la nota
  '''
  try:
    nota = requests.get(url_nota)
  except Exception as e:
    print("Error scrapeando URL", url_nota)
    print(e)
    return None

  if nota.status_code != 200:
    print(f"Error obteniendo la nota {url_nota}")
    print(f"status coder = {nota.status_code}")
    return None
  
  s_nota = BeautifulSoup(nota.text, 'html')
  nota_dict = obtener_info(s_nota)
  nota_dict['url'] = url_nota

  return nota_dict

In [4]:
def obtener_info(s_nota):
  '''
  Dada un objeto "soup" de la libreria bs4 de una nota regresa la informacion 
  scrapeada de la misma.

  Precondicion: ingresar como unico parametros un objeto string con la url de
  la nota.
  Poscondicion: devuelve un diccionario con la informacion de la nota
  '''
  nota_dict={}
  #Fecha
  fecha = s_nota.find('span', attrs = {'pubdate':'pubdate'})
  if fecha:
    nota_dict["fecha"] = fecha.get('datetime')
  else: nota_dict["fecha"] = None

  #Titulo
  titulo = s_nota.find('h1', class_='article-title')
  if titulo:
    nota_dict["titulo"] = titulo.text
  else: nota_dict["titulo"] = None
  
  #Volanta
  volanta = s_nota.find('h2', attrs = {'class':'article-prefix'})
  if volanta:
    nota_dict["volanta"] = volanta.text
  else: nota_dict["volanta"] = None
  
  #Copete
  copete = s_nota.find('div', attrs = {'class':'article-summary'})
  if copete:
    nota_dict["copete"] = copete.text
  else: nota_dict["copete"] = None

  #Autor
  autor = s_nota.find('div', attrs = {'class':'article-author'})
  if autor:
    nota_dict["autor"] = autor.text
  else: nota_dict["autor"] = None

  #Cuerpo
  cuerpo = s_nota.find('div', attrs = {'class':'article-text'}).find_all('p')
  if len(cuerpo)>10:
    nota_dict["cuerpo"] = cuerpo
  else: nota_dict["cuerpo"] = None
  
  '''  
  #Imagen
  media = s_nota.find('div', attrs = {'class':'article-main-media-image'})
  if media:
    media = media.find('div')
    if media:
      media = media.find_all('img')
  
  if media == None:
    nota_dict["imagen"] = None
  else:
    imagen = media[-1].get('data-src')
    try:
      imagen = requests.get(imagen)
    except Exception as e:
      print("Error scrapeando URL", imagen)
      print(e)
      return None
    if imagen.status_code != 200:
      print(f"Error obteniendo la nota {imagen}")
      print(f"status coder = {imagen.status_code}")
      return None
      
    nota_dict["imagen"] = imagen.content
    
    '''

  return nota_dict

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import datetime

df = ScrapearP12()

Scrapeando nota 0/67
Scrapeando nota 1/67
Scrapeando nota 2/67
Scrapeando nota 3/67
Scrapeando nota 4/67
Scrapeando nota 5/67
Scrapeando nota 6/67
Scrapeando nota 7/67
Scrapeando nota 8/67
Scrapeando nota 9/67
Scrapeando nota 10/67
Scrapeando nota 11/67
Scrapeando nota 12/67
Scrapeando nota 13/67
Scrapeando nota 14/67
Scrapeando nota 15/67
Scrapeando nota 16/67
Scrapeando nota 17/67
Scrapeando nota 18/67
Scrapeando nota 19/67
Scrapeando nota 20/67
Scrapeando nota 21/67
Scrapeando nota 22/67
Scrapeando nota 23/67
Scrapeando nota 24/67
Scrapeando nota 25/67
Scrapeando nota 26/67
Scrapeando nota 27/67
Scrapeando nota 28/67
Scrapeando nota 29/67
Scrapeando nota 30/67
Scrapeando nota 31/67
Scrapeando nota 32/67
Scrapeando nota 33/67
Scrapeando nota 34/67
Scrapeando nota 35/67
Scrapeando nota 36/67
Scrapeando nota 37/67
Scrapeando nota 38/67
Scrapeando nota 39/67
Scrapeando nota 40/67
Scrapeando nota 41/67
Scrapeando nota 42/67
Scrapeando nota 43/67
Scrapeando nota 44/67
Scrapeando nota 45/6

In [ ]:
df.head(5)